## Running a more complex circuit

A common building block of many quantum algorithms is the Quantum Fourier Transform (QFT), which transforms the basis states of an N-qubit system an integer $|n\rangle$ 
to a Fourier basis

$$
\text{QFT}|n\rangle = \frac{1}{2^N} \sum_{k=0}^{n-1} \omega_{2^N}^{nk} |k\rangle
$$

where $\omega_{2^N} = e^{2 \pi i / 2^N}$. This algorithm is already provided by Cirq and can be used directly in a circuit:

In [18]:
import cirq
import qsimcirq
import time

In [3]:
c = cirq.Circuit()

In [4]:
qubits = cirq.LineQubit.range(22)

In [5]:
c.append(cirq.qft(*qubits))

In [6]:
print(c)

0: ────qft───
       │
1: ────#2────
       │
2: ────#3────
       │
3: ────#4────
       │
4: ────#5────
       │
5: ────#6────
       │
6: ────#7────
       │
7: ────#8────
       │
8: ────#9────
       │
9: ────#10───
       │
10: ───#11───
       │
11: ───#12───
       │
12: ───#13───
       │
13: ───#14───
       │
14: ───#15───
       │
15: ───#16───
       │
16: ───#17───
       │
17: ───#18───
       │
18: ───#19───
       │
19: ───#20───
       │
20: ───#21───
       │
21: ───#22───


In [7]:
c.append(cirq.measure_each(*qubits))

In [8]:
print(c)

0: ────qft───M───
       │
1: ────#2────M───
       │
2: ────#3────M───
       │
3: ────#4────M───
       │
4: ────#5────M───
       │
5: ────#6────M───
       │
6: ────#7────M───
       │
7: ────#8────M───
       │
8: ────#9────M───
       │
9: ────#10───M───
       │
10: ───#11───M───
       │
11: ───#12───M───
       │
12: ───#13───M───
       │
13: ───#14───M───
       │
14: ───#15───M───
       │
15: ───#16───M───
       │
16: ───#17───M───
       │
17: ───#18───M───
       │
18: ───#19───M───
       │
19: ───#20───M───
       │
20: ───#21───M───
       │
21: ───#22───M───


Simulating this circuit with Cirq's native simulator is a bit time consuming...

In [17]:
#let's simulate the circuit with cirq native simulator
cirq_simulator = cirq.Simulator()
start_time = time.time()
cirq_simulator_result = cirq_simulator.sample(c, repetitions=20)
print(repr(cirq_simulator_result))
print("--- %s seconds ---" % (time.time() - start_time))
print("runtime: {:.2f}s".format(time.time() - start_time))


    0  1  2  3  4  5  6  7  8  9  ...  12  13  14  15  16  17  18  19  20  21
0   0  0  1  0  0  1  1  0  1  0  ...   0   1   1   0   1   1   1   0   0   0
1   1  0  1  1  1  1  1  0  1  0  ...   1   0   1   0   0   0   1   1   1   0
2   0  1  1  0  0  1  0  0  1  0  ...   0   1   1   1   1   1   0   1   0   1
3   1  1  1  1  0  0  1  0  1  1  ...   0   0   1   1   0   0   0   0   0   1
4   1  0  1  1  0  1  1  0  1  1  ...   1   0   0   1   1   0   0   1   0   0
5   0  1  1  1  0  0  1  0  0  1  ...   1   1   0   1   1   0   0   1   0   1
6   1  0  0  0  1  1  1  0  1  0  ...   0   0   1   0   1   0   1   0   1   0
7   1  1  1  0  1  0  0  0  0  0  ...   1   1   0   0   0   1   1   0   1   1
8   0  1  0  1  1  0  0  0  1  0  ...   0   1   1   1   1   1   1   1   0   0
9   1  0  1  1  0  1  1  1  0  0  ...   1   1   1   1   0   0   1   0   0   1
10  0  1  0  1  1  0  1  0  1  0  ...   1   1   1   1   0   1   0   0   1   1
11  1  1  0  0  0  0  1  1  1  0  ...   1   0   0   1   1   0   

## Running with qsim + cuQuantum

The same circuit simulated with qsim on GPUs using NVIDIA cuQuantum library can accelerate this simulation substantially.

We first need to setup the simulator with GPU support. The NVIDIA cuQuantum Appliance added a few options to QsimOptions:

#### gpu_mode
The GPU simulator backend to use. If 1, the simulator backend will use cuStateVec. If n, an integer greater than 1, the simulator will use the multi-GPU backend with the first n devices (if available). If a sequence of integers, the simulator will use the multi-GPU backend with devices whose ordinals match the values in the list. Default is to use the multi-GPU backend with device 0

#### use_sampler
If None, the multi-GPU backend will use its sampler, and all other backends will use their default sampler. If True, use the multi-GPU backend’s sampler. If False, the multi-GPU backend’s sampler is disabled

#### disable_gpu
Whether or not to disable the GPU simulator backend. All GPU options are only considered when this is False (default). Note the difference from qsimcirq’s use_gpu keyword.

We setup the qsim simulator with one GPU:

In [19]:
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
        max_fused_gate_size = 2
        , cpu_threads = 1
        , gpu_mode = ngpus
        , use_sampler = True
        , disable_gpu = False
    )
qsim_simulator = qsimcirq.QSimSimulator(qsim_options)

... and rerun the same circuit simulation on the GPU using the same syntax as before

In [20]:
#let's simulate the circuit with qsim cirq simulator
start_time = time.time()
qsim_simulator_result = qsim_simulator.sample(c, repetitions=20)
print(repr(qsim_simulator_result))
print("--- %s seconds ---" % (time.time() - start_time))
print("runtime: {:.2f}s".format(time.time() - start_time))

    0  1  2  3  4  5  6  7  8  9  ...  12  13  14  15  16  17  18  19  20  21
0   0  1  0  1  1  1  1  0  1  0  ...   0   0   0   0   1   0   1   1   0   0
1   1  1  0  0  0  0  0  1  0  0  ...   1   1   0   0   1   0   1   1   0   1
2   0  0  1  1  1  1  0  1  1  0  ...   1   0   0   0   0   1   1   0   1   1
3   1  0  0  1  1  1  1  1  0  1  ...   0   0   0   0   1   1   1   1   0   1
4   1  0  0  1  0  0  0  1  1  1  ...   1   1   0   0   1   0   1   0   1   1
5   1  1  1  0  0  0  1  0  1  0  ...   0   1   1   1   1   0   0   0   0   0
6   1  0  1  0  1  1  0  0  0  1  ...   0   0   1   0   0   0   1   1   0   1
7   0  0  0  1  1  1  1  1  0  0  ...   0   0   1   1   0   1   0   1   0   0
8   1  0  1  1  1  1  1  1  0  1  ...   0   0   0   0   0   0   0   1   1   1
9   0  1  1  1  0  1  0  0  0  0  ...   0   0   0   0   1   0   1   1   1   0
10  1  0  0  0  1  1  0  1  1  0  ...   0   1   1   1   1   1   0   0   0   1
11  1  0  0  1  1  1  1  1  1  0  ...   1   0   0   1   1   1   